# Back to the Basic (Part 2-2. Linear Regression)

* 2023.05.08.(Mon)
* Dept. of Math., Inha Univ.
* Byung Chun Kim (wizardbc@gmail.com)

## Linear Model

* A model is a mathematical function $f_{\theta}:X\rightarrow Y$ with some parameters $\theta$.
* We need:
  * a function $f$,
  * a way to set and get parameters,
  * a way to process a bunch (called a batch) of inputs.

In [ ]:
from typing import Iterable

class Module:
  def __init__(self):
    raise NotImplementedError

  def set_params(self) -> None:
    raise NotImplementedError
  
  def get_params(self) -> dict:
    raise NotImplementedError
  
  def f(self, x):
    raise NotImplementedError
  
  def __call__(self, x:Iterable) -> tuple:
    return tuple([self.f(t) for t in x])

* A linear model is consists of two parameters `weight` and `bias`. (aka `slope` and `intercept`, resp.)
* We will consider a simple linear function $f_{w,b}:\mathbb{R}\rightarrow\mathbb{R}$ defined by $f_{w,b}(x) = xw + b$.

In [ ]:
class LinearModel(Module):
  def __init__(self, w:float=.0, b:float=.0) -> None:
    self.set_params(w, b)

  def set_params(self, w:float, b:float) -> None:
    self.w = w
    self.b = b

  def get_params(self) -> dict[str,float]:
    return {'w': self.w, 'b':self.b}

  def f(self, x:float) -> float:
    params = self.get_params()
    w = params.get('w')
    b = params.get('b')
    return w * x + b

In [ ]:
x = [1,2,3,4,5]

lin = LinearModel(2.0, 1.0)
lin(x)

## Mean Squared Error and its Gradient

$f_\theta(x) = x w + b$ where $\theta=(w,b)$.

$\operatorname{MSE}_\theta(x, y) = \frac{1}{n}\sum^n_{i=1}(f(x_i) - y_i)^2 = \frac{1}{n}\sum^n_{i=1}(x_iw+b - y_i)^2$.

$$\frac{\partial}{\partial w} \operatorname{MSE}_{\theta}(x,y) = \frac{2}{n}\sum^n_{i=1}x_i(x_iw+b - y_i).$$

$$\frac{\partial}{\partial b} \operatorname{MSE}_{\theta}(x,y) = \frac{2}{n}\sum^n_{i=1}(x_iw+b - y_i).$$

In [ ]:
def mse(model:Module, x:Iterable[float], y_true:Iterable[float]) -> float:
  assert len(x) == len(y_true)
  n = len(x)
  y_pred = model(x)
  return sum([(t-p)**2 for t,p in zip(y_true, y_pred)])/n

def grad_mse(model:Module, x:Iterable[float], y_true:Iterable[float]) -> dict[str,float]:
  assert len(x) == len(y_true)
  n = len(x)
  y_pred = model(x)
  d_w = sum([x*(p-t) for x,p,t in zip(x, y_pred, y_true)])*2/n
  d_b = sum([(p-t) for p,t in zip(y_pred, y_true)])*2/n
  return {'d_w': d_w, 'd_b':d_b}

In [ ]:
mse(lin, [1.0], [1.0])

In [ ]:
grad_mse(lin, [1.0], [1.0])

## Gradient Descent

* Update $\theta$: $$\theta_{\textrm{new}} = \theta_{\textrm{old}} - \alpha \nabla\operatorname{MSE}_{\theta_{\textrm{old}}}(x,y)$$ where $\alpha>0$ is a learning rate.

In [ ]:
def update(model:Module, lr:float, d_w:float, d_b:float) -> None:
  params_old = model.get_params()
  params_new = {
    'w': params_old.get('w') - lr*d_w,
    'b': params_old.get('b') - lr*d_b,
  }
  model.set_params(**params_new)

## Data

In [ ]:
import random
import matplotlib.pyplot as plt

# dataset
# y = a*x + b
f = lambda x: 2.0*x + 1.0

xs = tuple([random.uniform(-1,1) for _ in range(100)])    # 100 points
ys = tuple([f(x)+0.1*random.gauss(0,1) for x in xs])


plt.title("Dataset")
plt.scatter(xs, ys, s=1)
plt.plot(xs, [f(x) for x in xs], label=f"y =  2x+1")
plt.legend()
plt.show()

In [ ]:
lin = LinearModel(0.,0.)
history = [lin.get_params()]

for epoch in range(200):
  grad = grad_mse(lin, xs, ys)
  update(lin, 0.1, **grad)
  err = mse(lin, xs, ys)
  params = lin.get_params()
  history.append(params)
  print(f"Epoch {epoch+1}: mse={err:.4f}, w={params.get('w'):.4f}, b={params.get('b'):.4f}")

In [ ]:
import plotly.express as px
import pandas as pd

df = pd.DataFrame(history, columns=['w','b'])
df = df.set_index(df.index.set_names('epoch')).reset_index()
df0 = df.copy()
df1 = df.copy()
df0['x'] = min(xs)
df1['x'] = max(xs)
df = pd.concat([df0, df1]).reset_index(drop=True)
df['y'] = df.w * df.x + df.b

fig = px.line(df, x='x', y='y', animation_frame="epoch", width=500, height=500)

fig.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 0.1
fig.layout.updatemenus[0].buttons[0].args[1]['transition']['duration'] = 0.1
fig.layout.updatemenus[0].buttons[0].args[1]['frame']['redraw'] = True

fig.add_scatter(x=xs, y=ys, mode='markers', name='data', marker={'size':2})

for i, frame in enumerate(fig.frames):
    frame['layout']['title_text'] = f"Prediction: y = {history[i]['w']:.4f}x{'' if history[i]['b'] < 0 else '+'}{history[i]['b']:.4f}"

fig.update_layout(template='plotly_dark')
fig.show()